In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor #biblioteca para rodar o código em paralelo

In [3]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.69'}

urls = ['https://unicode.org/emoji/charts/full-emoji-list.html','https://unicode.org/emoji/charts/full-emoji-modifiers.html']

unicode = []
emoji = []
cldr = []

for url in urls:

    response_text = requests.get(url= url, headers= headers).text

    soup = BeautifulSoup(response_text, 'lxml')

    main_table = soup.find('div', class_='main').find('table')
    lines = main_table.find_all('tr')

    for line in lines:
        try:
            unicode.append(line.find('td', class_='code').text)
            emoji.append(line.find('td', class_='chars').text)
            cldr.append(line.find('td', class_='name').text)

        except:
            continue

In [4]:
replacing_words = {
    ' ' : '-',
    ':' : '',
    '.' : '',
    '"' : '',
    ',' : '',
    '⊛' : '',
    "'" : '',
    "!" : '',
    '(' : '',
    ')' : '',
    '&' : '',
    'ç' : 'c',
    'é' : 'e',
    'í' : 'i',
    "’" : '',
    'ñ' : 'n',
    '“' : '',
    '”' : '',
    'å' : 'a',
    'ô' : 'o',
    'ã' : 'a',
    'keycap-#' : 'keycap-number-sign',
    'keycap-*' : 'keycap-asterisk',
    'keycap-0' : 'keycap-digit-zero',
    'keycap-1' : 'keycap-digit-one',
    'keycap-2' : 'keycap-digit-two',
    'keycap-3' : 'keycap-digit-three',
    'keycap-4' : 'keycap-digit-four',
    'keycap-5' : 'keycap-digit-five',
    'keycap-6' : 'keycap-digit-six',
    'keycap-7' : 'keycap-digit-seven',
    'keycap-8' : 'keycap-digit-eight',
    'keycap-9' : 'keycap-digit-nine',
    'red-circle' : 'large-red-circle',
    'orange-circle' : 'large-orange-circle',
    'yellow-circle' : 'large-yellow-circle',
    'green-circle' : 'large-green-circle',
    'blue-circle' : 'large-blue-circle',
    'purple-circle' : 'large-purple-circle',
    'brown-circle' : 'large-brown-circle',
    'smiling-face-with-open-hands':'hugging-face',
    'face-with-crossed-out-eyes' : 'dizzy-face',
    'enraged-face' : 'pouting-face',
    'ginger-root' : 'ginger',
    'two-oclock' : 'two-oclocktime',
    'bullseye' : 'direct-hit',
    'water-pistol' : 'pistol',
    'red-question-mark' : 'question-mark',
    'red-exclamation-mark' : 'exclamation-mark',
    'hollow-large-red-circle' : 'hollow-red-circle',
    'keycap-digit-one0' : 'keycap-10'

}

cldr_link = []
for word in cldr:
    word = word.lower().strip()
    for key in replacing_words:
        if key in word:
            word = word.replace(key, replacing_words[key])
    
    if word[0] == '-':
        word = word[1:]
    
    word = word.replace('---','-').replace("--","-")
    cldr_link.append(word)

cldr_link

['grinning-face',
 'grinning-face-with-big-eyes',
 'grinning-face-with-smiling-eyes',
 'beaming-face-with-smiling-eyes',
 'grinning-squinting-face',
 'grinning-face-with-sweat',
 'rolling-on-the-floor-laughing',
 'face-with-tears-of-joy',
 'slightly-smiling-face',
 'upside-down-face',
 'melting-face',
 'winking-face',
 'smiling-face-with-smiling-eyes',
 'smiling-face-with-halo',
 'smiling-face-with-hearts',
 'smiling-face-with-heart-eyes',
 'star-struck',
 'face-blowing-a-kiss',
 'kissing-face',
 'smiling-face',
 'kissing-face-with-closed-eyes',
 'kissing-face-with-smiling-eyes',
 'smiling-face-with-tear',
 'face-savoring-food',
 'face-with-tongue',
 'winking-face-with-tongue',
 'zany-face',
 'squinting-face-with-tongue',
 'money-mouth-face',
 'hugging-face',
 'face-with-hand-over-mouth',
 'face-with-open-eyes-and-hand-over-mouth',
 'face-with-peeking-eye',
 'shushing-face',
 'thinking-face',
 'saluting-face',
 'zipper-mouth-face',
 'face-with-raised-eyebrow',
 'neutral-face',
 'expres

In [5]:
cldr_link[3663]

'dark-skin-tone'

In [6]:
df_phase_1 = pd.DataFrame({'unicode': unicode, 'emoji': emoji, 'cldr': cldr, 'cldr_link': cldr_link})
df_phase_1.to_csv('df_phase_1.csv', index=False)

In [7]:
def make_request(link, header):
    
    time.sleep(3) 
    response = requests.get(link, headers=header)
    if response.status_code == 429:
        retry_after = int(response.headers.get("Retry-After", 30))  #Retry-After retorna quantos segundos é preciso esperar para 
        print("Received 429, sleeping for", retry_after, "seconds") #tentar novamente e não dar erro, caso não esteja especificado no header, 
        time.sleep(retry_after)                                     #considerar 30 segundos
        return make_request(link, header)
    return response

In [8]:
meanings = []
notes = []
unicodes = []

def crawler_emojipedia(link_cldr):
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.69'}

    url = f'https://emojipedia.org/{link_cldr}'

    response = make_request(link= url, header=headers)

    if response.status_code == 200:

        meaning_text = []
        note_text = []
        unicode_text = []
        
        response_text = response.text

        soup = BeautifulSoup(response_text, 'lxml')

        try:
            paragraphs = soup.find('div', class_='content').find('section', class_='description').find_all('p')

            note_str = ['Note:', 'ZWJ', 'Zero Width Joiner']
            unicode_str = ['was approved', 'was added to', 'as part of Unicode']
            for paragraph in paragraphs:

                p_text = paragraph.get_text(separator='')

                if any(word in p_text for word in note_str):
                    note_text.append(p_text.replace('\xa0',' ').replace('\n',''))

                elif any(word in p_text for word in unicode_str):
                    unicode_text.append(p_text.replace('\xa0',' ').replace('\n',''))
                
                else: 
                    meaning_text.append(p_text.replace('\xa0',' ').replace('\n',''))

            for elem in [meaning_text, note_text, unicode_text]:
                if len(elem) == 0:
                    elem.append('-')

            meanings.append(' '.join(meaning_text))
            notes.append(' '.join(note_text))
            unicodes.append(' '.join(unicode_text))

        except:
            print(f'erro em: {url}')
            meanings.append('-')
            notes.append('-')
            unicodes.append('-')

        finally:
            print(url)

    else: 
        print(f'erro por não ter achado a página em: {url}')
        meanings.append('-')
        notes.append('-')
        unicodes.append('-')
    
    return

#Parallel processing

with ThreadPoolExecutor() as executor:
    executor.map(crawler_emojipedia, cldr_link)

https://emojipedia.org/slightly-smiling-face
https://emojipedia.org/rolling-on-the-floor-laughing
https://emojipedia.org/grinning-face
https://emojipedia.org/grinning-face-with-big-eyes
https://emojipedia.org/grinning-face-with-sweat
https://emojipedia.org/winking-face
https://emojipedia.org/beaming-face-with-smiling-eyes
https://emojipedia.org/smiling-face-with-hearts
https://emojipedia.org/grinning-squinting-face
https://emojipedia.org/melting-face
https://emojipedia.org/smiling-face-with-smiling-eyes
https://emojipedia.org/face-with-tears-of-joy
https://emojipedia.org/upside-down-face
https://emojipedia.org/grinning-face-with-smiling-eyes
https://emojipedia.org/smiling-face-with-halo
https://emojipedia.org/smiling-face-with-heart-eyes
https://emojipedia.org/face-blowing-a-kiss
https://emojipedia.org/star-struck
https://emojipedia.org/kissing-face
https://emojipedia.org/smiling-face
https://emojipedia.org/smiling-face-with-tear
https://emojipedia.org/kissing-face-with-smiling-eyes
ht

In [9]:
print(len(meanings),len(notes),len(unicodes))

3664 3664 3664


In [10]:
emojipedia =pd.DataFrame(data={'cldr_link': cldr_link, 'meaning':meanings,'note':notes,'unicode_text':unicodes})

In [12]:
df_phase_2_emojipedia = df_phase_1.merge(emojipedia, how='inner', on='cldr_link')

In [13]:
df_phase_2_emojipedia.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3664 entries, 0 to 3663
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   unicode       3664 non-null   object
 1   emoji         3664 non-null   object
 2   cldr          3664 non-null   object
 3   cldr_link     3664 non-null   object
 4   meaning       3664 non-null   object
 5   note          3664 non-null   object
 6   unicode_text  3664 non-null   object
dtypes: object(7)
memory usage: 229.0+ KB


In [15]:
df_phase_2_emojipedia.to_csv('df_phase_2_emojipedia.csv', index=False)

In [16]:
url_dictionary = []
name_dictionary = []

for i in range(1,8):
    params = {
        'termId': '7842',
        'postsPerPage': '60',
        'page': f'{i}',
        'action': 'get_word_category_page',
        'nonce': '16022f1552',
    }

    response0 = requests.get('https://www.dictionary.com/e/crb-ajax/cached.php', params=params).json()
    for j in range(len(response0['data']['words'])):
        name_dictionary.append(response0['data']['words'][j]['name'])
        url_dictionary.append(response0['data']['words'][j]['permalink'])

In [17]:
len(url_dictionary)

386

In [18]:
df_phase_2_dictionary = pd.DataFrame({'name': name_dictionary, 'link': url_dictionary})
df_phase_2_dictionary

,name,link
0,💩 Pile of Poo emoji,https://www.dictionary.com/e/emoji/pile-of-poo...
1,🇺🇦 Flag of Ukraine emoji,https://www.dictionary.com/e/emoji/flag-of-ukr...
2,🌻 Sunflower emoji,https://www.dictionary.com/e/emoji/sunflower-e...
3,⛎ Ophiuchus Emoji,https://www.dictionary.com/e/emoji/ophiuchus-e...
4,♉ Taurus Emoji,https://www.dictionary.com/e/emoji/taurus-emoji/
...,...,...
381,🙄 Face With Rolling Eyes emoji,https://www.dictionary.com/e/emoji/face-with-r...
382,😂 Face With Tears of Joy emoji,https://www.dictionary.com/e/emoji/face-with-t...
383,🔥 Fire emoji,https://www.dictionary.com/e/emoji/fire-emoji/
384,🙏 Folded Hands emoji,https://www.dictionary.com/e/emoji/folded-hand...


In [19]:
def find_next_char(string, word): #Function to find the next character after the word "and", if it's a emoji so the name contain more than 1 emoji
    index = string.find(word)
    if index != -1 and index+len(word) < len(string):
        next_char = string[index+len(word)]
        return next_char
    else:
        return False

In [20]:
def drop_line(string): #Function to know which line will be droped according to the conditions
    if ord(string[0]) < 591:
        return True 
    elif string.find(" and ") != -1:
        if ord(find_next_char(string, ' and ')) > 591:
            return True
        else:
            return False
    else:
        return False

In [21]:
df_phase_2_dictionary['drop'] = df_phase_2_dictionary['name'].apply(lambda x: drop_line(x))
df_phase_2_dictionary

,name,link,drop
0,💩 Pile of Poo emoji,https://www.dictionary.com/e/emoji/pile-of-poo...,False
1,🇺🇦 Flag of Ukraine emoji,https://www.dictionary.com/e/emoji/flag-of-ukr...,False
2,🌻 Sunflower emoji,https://www.dictionary.com/e/emoji/sunflower-e...,False
3,⛎ Ophiuchus Emoji,https://www.dictionary.com/e/emoji/ophiuchus-e...,False
4,♉ Taurus Emoji,https://www.dictionary.com/e/emoji/taurus-emoji/,False
...,...,...,...
381,🙄 Face With Rolling Eyes emoji,https://www.dictionary.com/e/emoji/face-with-r...,False
382,😂 Face With Tears of Joy emoji,https://www.dictionary.com/e/emoji/face-with-t...,False
383,🔥 Fire emoji,https://www.dictionary.com/e/emoji/fire-emoji/,False
384,🙏 Folded Hands emoji,https://www.dictionary.com/e/emoji/folded-hand...,False


In [22]:
df_phase_2_dictionary = df_phase_2_dictionary.drop(index=df_phase_2_dictionary.loc[df_phase_2_dictionary['drop'] == True].index, axis=0)

In [23]:
df_phase_2_dictionary = df_phase_2_dictionary.reset_index(drop=True)

In [24]:
df_phase_2_dictionary = df_phase_2_dictionary.drop(columns='drop', axis=1)

In [25]:
df_phase_2_dictionary

,name,link
0,💩 Pile of Poo emoji,https://www.dictionary.com/e/emoji/pile-of-poo...
1,🇺🇦 Flag of Ukraine emoji,https://www.dictionary.com/e/emoji/flag-of-ukr...
2,🌻 Sunflower emoji,https://www.dictionary.com/e/emoji/sunflower-e...
3,⛎ Ophiuchus Emoji,https://www.dictionary.com/e/emoji/ophiuchus-e...
4,♉ Taurus Emoji,https://www.dictionary.com/e/emoji/taurus-emoji/
...,...,...
367,🙄 Face With Rolling Eyes emoji,https://www.dictionary.com/e/emoji/face-with-r...
368,😂 Face With Tears of Joy emoji,https://www.dictionary.com/e/emoji/face-with-t...
369,🔥 Fire emoji,https://www.dictionary.com/e/emoji/fire-emoji/
370,🙏 Folded Hands emoji,https://www.dictionary.com/e/emoji/folded-hand...


In [26]:
link_list_dictionary = df_phase_2_dictionary.link.to_list()
link_list_dictionary

['https://www.dictionary.com/e/emoji/pile-of-poo-emoji/',
 'https://www.dictionary.com/e/emoji/flag-of-ukraine-emoji/',
 'https://www.dictionary.com/e/emoji/sunflower-emoji/',
 'https://www.dictionary.com/e/emoji/ophiuchus-emoji/',
 'https://www.dictionary.com/e/emoji/taurus-emoji/',
 'https://www.dictionary.com/e/emoji/aries-emoji/',
 'https://www.dictionary.com/e/emoji/pisces-emoji/',
 'https://www.dictionary.com/e/emoji/fish-emoji/',
 'https://www.dictionary.com/e/emoji/heart-with-arrow-emoji/',
 'https://www.dictionary.com/e/emoji/curling-stone-emoji/',
 'https://www.dictionary.com/e/emoji/tiger-face-emoji/',
 'https://www.dictionary.com/e/emoji/tiger-emoji/',
 'https://www.dictionary.com/e/emoji/aquarius-emoji/',
 'https://www.dictionary.com/e/emoji/capricorn-emoji/',
 'https://www.dictionary.com/e/emoji/mrs-claus/',
 'https://www.dictionary.com/e/emoji/star-of-david-emoji/',
 'https://www.dictionary.com/e/emoji/menorah-emoji/',
 'https://www.dictionary.com/e/emoji/cookie-emoji/',

In [27]:
definition = []
origin = []


def crawler_emoji_dictionary(link):

    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.69'}

    response = make_request(link=link, header=headers)
    if response.status_code == 200:
        response_text = response.text

        meaning_paragraph = []
        origin_paragraph = []

        soup = BeautifulSoup(response_text, 'lxml')
        
        try:
            meaning_dict = soup.find('div', class_='article-word__definition').find_all('p')
           
            for mean in meaning_dict:
                meaning_paragraph.append(mean.get_text(separator='').replace('\xa0',' ').replace('\n',''))

        except:
                meaning_paragraph.append('-')
        
        definition.append(' '.join(meaning_paragraph))
        
        try:
            origin_dict = soup.find('div', class_='article-word__origin-content').div.find_next_siblings('p')

            for orig in origin_dict:
                origin_paragraph.append(orig.get_text(separator='').replace('\xa0',' ').replace('\n',''))      
                
        except:
                origin_paragraph.append('-')
        
        origin.append(' '.join(origin_paragraph))

    else: #tratamento de erro para por não encontrar a página
        print(f'página não encontrada: {link}')
        definition.append('-')
        origin.append('-')
    
    return

#Rodar em paralelo

with ThreadPoolExecutor() as executor:
    executor.map(crawler_emoji_dictionary, link_list_dictionary)


In [28]:
df_phase_2_dictionary = df_phase_2_dictionary.merge(pd.DataFrame(data = {'link' : link_list_dictionary, 'definition' : definition, 'origin' : origin}), how='inner', on='link')

In [29]:
df_phase_2_dictionary.to_csv('df_phase_2_dictionary.csv', index=False)

In [30]:
df_phase_2_dictionary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 372 entries, 0 to 371
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        372 non-null    object
 1   link        372 non-null    object
 2   definition  372 non-null    object
 3   origin      372 non-null    object
dtypes: object(4)
memory usage: 14.5+ KB
